In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/WA_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5791407702,WA,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,5.303301e+14
1,1101807307,WA,"4526 FEDERAL AVE Building 4E, EVERETT, WA",47.957662,-122.218801,5.306104e+14
2,1351807201,WA,"18815 139th Ave NE Suite C, WOODINVILLE, WA",47.764576,-122.155385,5.303303e+14
3,2545378910,WA,"921 Terry Ave, Seattle, WA",47.608549,-122.326329,5.303301e+14
4,2664207106,WA,"2035 158th Court NE 0.0, Bellevue, WA",47.629101,-122.128750,5.303302e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5791407702,WA,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,5.303301e+14
1,1101807307,WA,"4526 FEDERAL AVE Building 4E, EVERETT, WA",47.957662,-122.218801,5.306104e+14
2,1351807201,WA,"18815 139th Ave NE Suite C, WOODINVILLE, WA",47.764576,-122.155385,5.303303e+14
3,2545378910,WA,"921 Terry Ave, Seattle, WA",47.608549,-122.326329,5.303301e+14
4,2664207106,WA,"2035 158th Court NE 0.0, Bellevue, WA",47.629101,-122.128750,5.303302e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
735,6610757306,WA,"3405 Lind Avenue SW, RENTON, WA",47.447951,-122.224241,NaN
1315,3944228804,WA,"11010 NE 8TH ST STE 465, BELLEVUE, WA",47.617876,-122.193094,NaN
4191,5492258309,WA,"4205 9th Ave NW, Seattle, WA",47.658459,-122.368059,NaN
5738,9140668700,WA,"11010 NE 8TH ST STE 465, BELLEVUE, WA",47.617876,-122.193094,NaN
5990,5419248800,WA,"11010 NE 8TH ST STE 465, BELLEVUE, WA",47.617876,-122.193094,NaN
8193,7040088805,WA,"11010 NE 8TH ST STE 465, BELLEVUE, WA",47.617876,-122.193094,NaN
8632,3731107204,WA,"2213 Main Street, VANCOUVER, WA",45.638216,-122.671203,NaN
8678,9051167008,WA,"11010 NE 8TH ST STE 465, BELLEVUE, WA",47.617876,-122.193094,NaN
9169,2388118505,WA,"8608 S March Point Rd, Anacortes, WA",48.460929,-122.559806,NaN
10152,5501127410,WA,"4730 University Way NE Suite 104-98172, SEATTL...",47.663958,-122.312819,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('53')] #WA
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/30 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

30


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_11101/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
735,6610757306,WA,"3405 Lind Avenue SW, RENTON, WA",47.447951,-122.224241,[530330262001119]
1315,3944228804,WA,"11010 NE 8TH ST STE 465, BELLEVUE, WA",47.617876,-122.193094,[530330238062003]
4191,5492258309,WA,"4205 9th Ave NW, Seattle, WA",47.658459,-122.368059,[530330047033026]
5738,9140668700,WA,"11010 NE 8TH ST STE 465, BELLEVUE, WA",47.617876,-122.193094,[530330238062003]
5990,5419248800,WA,"11010 NE 8TH ST STE 465, BELLEVUE, WA",47.617876,-122.193094,[530330238062003]
8193,7040088805,WA,"11010 NE 8TH ST STE 465, BELLEVUE, WA",47.617876,-122.193094,[530330238062003]
8632,3731107204,WA,"2213 Main Street, VANCOUVER, WA",45.638216,-122.671203,[530110425001008]
8678,9051167008,WA,"11010 NE 8TH ST STE 465, BELLEVUE, WA",47.617876,-122.193094,[530330238062003]
9169,2388118505,WA,"8608 S March Point Rd, Anacortes, WA",48.460929,-122.559806,[530579501002025]
10152,5501127410,WA,"4730 University Way NE Suite 104-98172, SEATTL...",47.663958,-122.312819,[530330053061001]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_11101/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
735,6610757306,WA,"3405 Lind Avenue SW, RENTON, WA",47.447951,-122.224241,530330262001119
1315,3944228804,WA,"11010 NE 8TH ST STE 465, BELLEVUE, WA",47.617876,-122.193094,530330238062003
4191,5492258309,WA,"4205 9th Ave NW, Seattle, WA",47.658459,-122.368059,530330047033026
5738,9140668700,WA,"11010 NE 8TH ST STE 465, BELLEVUE, WA",47.617876,-122.193094,530330238062003
5990,5419248800,WA,"11010 NE 8TH ST STE 465, BELLEVUE, WA",47.617876,-122.193094,530330238062003
8193,7040088805,WA,"11010 NE 8TH ST STE 465, BELLEVUE, WA",47.617876,-122.193094,530330238062003
8632,3731107204,WA,"2213 Main Street, VANCOUVER, WA",45.638216,-122.671203,530110425001008
8678,9051167008,WA,"11010 NE 8TH ST STE 465, BELLEVUE, WA",47.617876,-122.193094,530330238062003
9169,2388118505,WA,"8608 S March Point Rd, Anacortes, WA",48.460929,-122.559806,530579501002025
10152,5501127410,WA,"4730 University Way NE Suite 104-98172, SEATTL...",47.663958,-122.312819,530330053061001


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
12059,9743267004,WA,"503 S. Hilltop Lane, FAIRFIELD, WA",39.012144,-89.801411,NaN
19361,6790008405,WA,"515 KUWILI ST UNIT E, HONOLULU, WA",21.317822,-157.868017,NaN
20192,2243158309,WA,"8960 STATE ROUTE 2 STE 146, Monroe, WA",45.897426,-86.912018,NaN
20198,5003828303,WA,"4 N Park St, Saint John, WA",43.067929,-89.401266,NaN
22745,9978467707,WA,"1316 J STREET, WASCO, WA",35.587474,-119.331151,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
735,6610757306,530330262001119
1315,3944228804,530330238062003
4191,5492258309,530330047033026
5738,9140668700,530330238062003
5990,5419248800,530330238062003
8193,7040088805,530330238062003
8632,3731107204,530110425001008
8678,9051167008,530330238062003
9169,2388118505,530579501002025
10152,5501127410,530330053061001


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/WA_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,919076,5791407702,WA,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,5.303301e+14
1,919077,1101807307,WA,"4526 FEDERAL AVE Building 4E, EVERETT, WA",47.957662,-122.218801,5.306104e+14
2,919078,1351807201,WA,"18815 139th Ave NE Suite C, WOODINVILLE, WA",47.764576,-122.155385,5.303303e+14
3,919079,2545378910,WA,"921 Terry Ave, Seattle, WA",47.608549,-122.326329,5.303301e+14
4,919080,2664207106,WA,"2035 158th Court NE 0.0, Bellevue, WA",47.629101,-122.128750,5.303302e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,919076,5791407702,WA,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,5.303301e+14,NaN
1,919077,1101807307,WA,"4526 FEDERAL AVE Building 4E, EVERETT, WA",47.957662,-122.218801,5.306104e+14,NaN
2,919078,1351807201,WA,"18815 139th Ave NE Suite C, WOODINVILLE, WA",47.764576,-122.155385,5.303303e+14,NaN
3,919079,2545378910,WA,"921 Terry Ave, Seattle, WA",47.608549,-122.326329,5.303301e+14,NaN
4,919080,2664207106,WA,"2035 158th Court NE 0.0, Bellevue, WA",47.629101,-122.128750,5.303302e+14,NaN
...,...,...,...,...,...,...,...,...
23615,942691,9825088608,WA,"6346 Rainier Ave S, Seattle, WA",47.545071,-122.274353,5.303301e+14,NaN
23616,942692,9830848507,WA,"6021 199th St NE Ste A, Arlington, WA",48.177036,-122.147012,5.306105e+14,NaN
23617,942693,9847407101,WA,"2145 hwy 101, PORT ANGELES, WA",48.106684,-123.398364,5.300900e+14,NaN
23618,942694,9939018302,WA,"4720 158th Pl SW, Lynnwood, WA",47.855326,-122.297679,5.306105e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,919076,5791407702,WA,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,5.303301e+14,NaN,530330062002034.0
1,919077,1101807307,WA,"4526 FEDERAL AVE Building 4E, EVERETT, WA",47.957662,-122.218801,5.306104e+14,NaN,530610410004001.0
2,919078,1351807201,WA,"18815 139th Ave NE Suite C, WOODINVILLE, WA",47.764576,-122.155385,5.303303e+14,NaN,530330323193008.0
3,919079,2545378910,WA,"921 Terry Ave, Seattle, WA",47.608549,-122.326329,5.303301e+14,NaN,530330083001012.0
4,919080,2664207106,WA,"2035 158th Court NE 0.0, Bellevue, WA",47.629101,-122.128750,5.303302e+14,NaN,530330230003002.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,5791407702,WA,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,530330062002034.0
1,1101807307,WA,"4526 FEDERAL AVE Building 4E, EVERETT, WA",47.957662,-122.218801,530610410004001.0
2,1351807201,WA,"18815 139th Ave NE Suite C, WOODINVILLE, WA",47.764576,-122.155385,530330323193008.0
3,2545378910,WA,"921 Terry Ave, Seattle, WA",47.608549,-122.326329,530330083001012.0
4,2664207106,WA,"2035 158th Court NE 0.0, Bellevue, WA",47.629101,-122.128750,530330230003002.0
...,...,...,...,...,...,...
23615,9825088608,WA,"6346 Rainier Ave S, Seattle, WA",47.545071,-122.274353,530330111022002.0
23616,9830848507,WA,"6021 199th St NE Ste A, Arlington, WA",48.177036,-122.147012,530610535091000.0
23617,9847407101,WA,"2145 hwy 101, PORT ANGELES, WA",48.106684,-123.398364,530090012003011.0
23618,9939018302,WA,"4720 158th Pl SW, Lynnwood, WA",47.855326,-122.297679,530610501023001.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
12059,9743267004,WA,"503 S. Hilltop Lane, FAIRFIELD, WA",39.012144,-89.801411,NaN
19361,6790008405,WA,"515 KUWILI ST UNIT E, HONOLULU, WA",21.317822,-157.868017,NaN
20192,2243158309,WA,"8960 STATE ROUTE 2 STE 146, Monroe, WA",45.897426,-86.912018,NaN
20198,5003828303,WA,"4 N Park St, Saint John, WA",43.067929,-89.401266,NaN
22745,9978467707,WA,"1316 J STREET, WASCO, WA",35.587474,-119.331151,NaN


Convert to File

In [19]:
fips_WA = fips_merge_drop

In [20]:
fips_WA.to_csv("../../../data/state_data/geo/geo_fips/23/WA_fips_scraped.csv") 